In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel("data\\data.xlsx")
df = df[df["LMTD"] > 0]
df = df[df["LA Chilling Duty"] >= 0]
df["SCD (MJ/kg)"] = (0.001 * df["LA Chilling Duty"]) / df["Captured CO2 (kg/h)"]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from joblib import dump, load

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import BayesianOptimization
from keras_tuner.tuners import Hyperband
import keras_tuner as kt

In [5]:
XA = df[["FG Temperature", "Lean Loading",'Lean Amine Temperature','L/G (kg/kg)','Flue gas CO2 cons','CO2 Capture Efficiency','LMTD','Abs Height']]
XN = df[["FG Temperature", "Lean Loading",'Lean Amine Temperature','L/G (kg/kg)','Flue gas CO2 cons','CO2 Capture Efficiency']]
y = df[["SRD (MJ/kg CO2)", "Condensor Specific Duty", "SCD (MJ/kg)"]]

In [6]:
XA_train, XA_test, y_train, y_test = train_test_split(XA, y, test_size=0.30, random_state=211)
XN_train, XN_test, y_train, y_test = train_test_split(XN, y, test_size=0.30, random_state=211)

In [7]:
XA_train_u, XA_validate, y_train_u, y_validate = train_test_split(XA_train, y_train, test_size=0.20, random_state=211)
XN_train_u, XN_validate, y_train_u, y_validate = train_test_split(XN_train, y_train, test_size=0.20, random_state=211)

In [8]:
scalerA = StandardScaler()
scalerA.fit(XA_train_u)
XA_train_u_s = scalerA.transform(XA_train_u)
XA_validate_s = scalerA.transform(XA_validate)

In [9]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(XA_train_u_s.shape[1],)))
    for l in range(hp.Int(name='n_layers', min_value=1, max_value=7)):
        model.add(layers.Dense(units=hp.Int(name='units_'+str(l),
                                           min_value=10,
                                           max_value=32,
                                           step=4
                                           ),
                              activation=hp.Choice(name='act_func', values=['relu', 'tanh'])
                              )
                 )
        
    model.add(layers.Dense(units=3, activation='relu'))
        
    model.compile(optimizer='adam',
                      loss=keras.losses.MeanSquaredError(),
                      metrics=keras.metrics.MeanSquaredError()
                     )
    return model

In [10]:
tuner = Hyperband(build_model,
                  objective='val_loss',
                  max_epochs=150,
                  executions_per_trial=2,
                  directory='DNN_HPFT',
                  overwrite=True
                            )

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 7, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 32, 'step': 4, 'sampling': None}
act_func (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}


In [12]:
tuner.search(XA_train_u_s, y_train_u,
             epochs=150,
             validation_data=(XA_validate_s, y_validate),
             verbose=1
            )

Trial 254 Complete [00h 00m 37s]
val_loss: 0.0024876389652490616

Best val_loss So Far: 0.002370692207477987
Total elapsed time: 00h 18m 54s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

Results summary
Results in DNN_HPFT\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
n_layers: 5
units_0: 22
act_func: tanh
units_1: 26
units_2: 30
units_3: 30
units_4: 30
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0142
Score: 0.002370692207477987
Trial summary
Hyperparameters:
n_layers: 7
units_0: 14
act_func: tanh
units_1: 26
units_2: 14
units_3: 22
units_4: 30
units_5: 14
units_6: 22
tuner/epochs: 150
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.0024876389652490616
Trial summary
Hyperparameters:
n_layers: 5
units_0: 30
act_func: tanh
units_1: 18
units_2: 30
units_3: 30
units_4: 18
units_5: 26
units_6: 30
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0244
Score: 0.0025431375252082944
Trial summary
Hyperparameters:
n_layers: 2
units_0: 30
act_func: tanh
units_1: 22
units_2: 26
units_3: 22
units_4: 18
units_5: 26
units_6: 14
tuner/epochs: 150
tuner/initial

In [14]:
tuner.get_best_hyperparameters()[0].values

{'n_layers': 5,
 'units_0': 22,
 'act_func': 'tanh',
 'units_1': 26,
 'units_2': 30,
 'units_3': 30,
 'units_4': 30,
 'tuner/epochs': 150,
 'tuner/initial_epoch': 50,
 'tuner/bracket': 4,
 'tuner/round': 4,
 'tuner/trial_id': '0142'}

In [15]:
gbmmodel = tuner.get_best_models()

In [16]:
gbmmodel[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 22)                198       
                                                                 
 dense_1 (Dense)             (None, 26)                598       
                                                                 
 dense_2 (Dense)             (None, 30)                810       
                                                                 
 dense_3 (Dense)             (None, 30)                930       
                                                                 
 dense_4 (Dense)             (None, 30)                930       
                                                                 
 dense_5 (Dense)             (None, 3)                 93        
                                                                 
Total params: 3,559
Trainable params: 3,559
Non-trainabl

In [17]:
final_pred = gbmmodel[0].predict(scalerA.transform(XA_test))

54/54 [==============================] - 0s 555us/step


In [18]:
final_pred

array([[4.1328406 , 1.4750441 , 0.3900515 ],
       [5.7145147 , 3.2218363 , 0.35100046],
       [3.9546764 , 1.2852015 , 0.5459489 ],
       ...,
       [3.4064562 , 1.4045744 , 0.31407675],
       [4.2017727 , 1.6695863 , 0.53281265],
       [3.800823  , 1.4076563 , 0.582796  ]], dtype=float32)

In [19]:
y_test['SRD (MJ/kg CO2)'].mean()

4.786278037383177

In [20]:
np.sqrt(mean_squared_error(y_test['SRD (MJ/kg CO2)'],final_pred[:,0]))

0.05220040166405218

In [22]:
mean_absolute_error(y_test['SRD (MJ/kg CO2)'],final_pred[:,0])

0.03899270320050072